# A. WORKING WITH TEXT DATA

In [1]:
# Check package version
from importlib.metadata import version

print("Torch version:", version("torch"))
print("Tiktoken version:", version("tiktoken"))

Torch version: 2.8.0+cu126
Tiktoken version: 0.11.0


In [2]:
import os
import urllib.request

# Download a sample text file
if not os.path.exists("the-verdict.txt"):
    url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
    urllib.request.urlretrieve(url, "the-verdict.txt")

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
import re

text = "Hello, World! This is a test text"
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'World!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', ' ', 'text']


In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'World!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', ' ', 'text']


In [6]:
# Strip whitespace from each item and then filter out any empty strings
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'World!', 'This', 'is', 'a', 'test', 'text']


In [7]:
text = "Hello, World! Is this-- a test text?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'World', '!', 'Is', 'this', '--', 'a', 'test', 'text', '?']


In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [9]:
print(len(preprocessed))

4690


# =====================================================================

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print("Vocab size:", vocab_size)

Vocab size: 1130


In [11]:
vocab = {token:integer for integer, token in enumerate(all_words)}

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        text = re.sub(r'\s([,.:;?_!"()\'])', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

test_text = "It's the last he dropped, you know, Mr. Greek said."
ids = tokenizer.encode(test_text)
print(ids)

[56, 2, 850, 988, 602, 533, 370, 5, 1126, 596, 5, 66, 7, 41, 851, 7]


In [15]:
tokenizer.decode(ids)
tokenizer.decode(tokenizer.encode(test_text))

"It' s the last he dropped, you know, Mr. Greek said."

# =====================================================================

In [16]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like coffee. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token: integer for integer, token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

1132

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        text = re.sub(r'\s([,.:;?_!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like coffee. Is this-- a test?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join([text1, text2])

print(text)

Hello, do you like coffee. Is this-- a test? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer.encode(text)

[1131,
 5,
 355,
 1126,
 628,
 1131,
 7,
 1131,
 999,
 6,
 115,
 1131,
 10,
 1130,
 55,
 988,
 956,
 984,
 722,
 988,
 1131,
 7]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like <|unk|>. <|unk|> this -- a <|unk|>? <|endoftext|> In the sunlit terraces of the <|unk|>.'

# =====================================================================

In [17]:
import importlib
import tiktoken

print("titoken version: ", importlib.metadata.version("tiktoken"))

titoken version:  0.11.0


In [18]:
tokenizer = tiktoken.get_encoding("gpt2")

In [19]:
text = (
    "Hello, do you like coffee. <|endoftext|> Is the sunlit terraces"
    "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 6891, 13, 220, 50256, 1148, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [20]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like coffee. <|endoftext|> Is the sunlit terracesof someunknownPlace.


# =====================================================================

In [21]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [22]:
enc_sample = enc_text[50:]

In [23]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f'x: {x}')
print(f'y:      {y}')

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [25]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(f'{context} -> {desired}')

[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [26]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(f'{tokenizer.decode(context)} -> {tokenizer.decode([desired])}')

 and ->  established
 and established ->  himself
 and established himself ->  in
 and established himself in ->  a
